***USE ad_viz_plotval_data.csv UPLOADED ON GITHUB ***

In [1]:
# Install required packages
!pip install folium
!pip install pandas
!pip install numpy

In [2]:
# Importing necessary libraries
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import pandas as pd
import numpy as np
from google.colab import files

In [3]:
# Upload the file manually
uploaded = files.upload()

Saving ad_viz_plotval_data.csv to ad_viz_plotval_data.csv


In [4]:
# Read the uploaded CSV file
data = pd.read_csv("ad_viz_plotval_data.csv")

# Display the first few rows of the data
data.head()

,Date,AQS_SITE_ID,POC,Daily Mean PM2.5 Concentration,UNITS,DAILY_AQI_VALUE,DAILY_OBS_COUNT,PERCENT_COMPLETE,AQS_PARAMETER_CODE,AQS_PARAMETER_DESC,CBSA_CODE,CBSA_NAME,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
0,01/04/2004,20900010,1,14.6,ug/m3 LC,56,1,100.0,88101,PM2.5 - Local Conditions,21820,"Fairbanks, AK",2,Alaska,90,Fairbanks North Star,64.840672,-147.722461
1,01/10/2004,20900010,1,14.4,ug/m3 LC,56,1,100.0,88101,PM2.5 - Local Conditions,21820,"Fairbanks, AK",2,Alaska,90,Fairbanks North Star,64.840672,-147.722461
2,01/16/2004,20900010,1,54.2,ug/m3 LC,147,1,100.0,88101,PM2.5 - Local Conditions,21820,"Fairbanks, AK",2,Alaska,90,Fairbanks North Star,64.840672,-147.722461
3,01/22/2004,20900010,1,11.1,ug/m3 LC,46,1,100.0,88101,PM2.5 - Local Conditions,21820,"Fairbanks, AK",2,Alaska,90,Fairbanks North Star,64.840672,-147.722461
4,01/28/2004,20900010,1,25.5,ug/m3 LC,79,1,100.0,88101,PM2.5 - Local Conditions,21820,"Fairbanks, AK",2,Alaska,90,Fairbanks North Star,64.840672,-147.722461


In [7]:
# Convert the Date column to datetime format and extract the week number
data["Date"] = pd.to_datetime(data["Date"])
data["Week"] = data["Date"].dt.isocalendar().week

In [9]:
# Calculate the weekly mean AQI value per location
weekly_mean_aqi = data.groupby(["AQS_SITE_ID", "Week"]).agg({
    'SITE_LATITUDE': 'first',
    'SITE_LONGITUDE': 'first',
    'DAILY_AQI_VALUE': lambda x: np.mean(x)
}).reset_index()

In [10]:
# Print weekly mean AQI value for each site
print("Weekly Mean AQI Value per Site:")
for site in weekly_mean_aqi["AQS_SITE_ID"].unique():
    site_data = weekly_mean_aqi[weekly_mean_aqi["AQS_SITE_ID"] == site]
    print(f"Site ID: {site}")
    for index, row in site_data.iterrows():
        print(f"Week: {row['Week']}, Average AQI Value: {row['DAILY_AQI_VALUE']:.2f}")
    print("\n")

Weekly Mean AQI Value per Site:
Site ID: 20900010
Week: 1, Average AQI Value: 56.00
Week: 2, Average AQI Value: 56.00
Week: 3, Average AQI Value: 147.00
Week: 4, Average AQI Value: 46.00
Week: 5, Average AQI Value: 79.00
Week: 6, Average AQI Value: 127.00
Week: 7, Average AQI Value: 56.50
Week: 8, Average AQI Value: 86.00
Week: 9, Average AQI Value: 76.00
Week: 10, Average AQI Value: 31.00
Week: 11, Average AQI Value: 49.00
Week: 12, Average AQI Value: 48.00
Week: 13, Average AQI Value: 41.50
Week: 14, Average AQI Value: 20.00
Week: 15, Average AQI Value: 31.00
Week: 16, Average AQI Value: 32.00
Week: 17, Average AQI Value: 13.00
Week: 18, Average AQI Value: 11.00
Week: 19, Average AQI Value: 15.50
Week: 21, Average AQI Value: 23.00
Week: 22, Average AQI Value: 13.00
Week: 23, Average AQI Value: 13.00
Week: 24, Average AQI Value: 13.00
Week: 25, Average AQI Value: 51.00
Week: 26, Average AQI Value: 51.00
Week: 27, Average AQI Value: 467.33
Week: 28, Average AQI Value: 35.00
Week: 29, A

In [12]:
# Prepare data for HeatMapWithTime
df_week_list = []
for week in range(1, 10):  # Adjust the range based on your data
    weekly_data = weekly_mean_aqi[weekly_mean_aqi["Week"] == week][['SITE_LATITUDE', 'SITE_LONGITUDE', 'DAILY_AQI_VALUE']]
    df_week_list.append(weekly_data.values.tolist())

# Create the base map centered on a specific location (adjust coordinates as needed)
base_map = folium.Map(location=[34.0549, -118.2426], zoom_start=11)

# Add the HeatMapWithTime to the map
HeatMapWithTime(df_week_list, radius=70, gradient={0.05: 'blue', 0.5: 'green', 0.75: 'yellow', 1.0: 'red'},
                min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)

# Save the map to an HTML file
base_map.save("weekly_aqi_heatmap.html")

In [13]:
# Display the map
base_map